# SiLU / Swish — Matematik, Türev ve Backprop 

Bu defter, **SiLU (Swish)** aktivasyonunun matematiğini **okunur** ve **neden–sonuç** ilişkisi kuracak şekilde anlatır.

Kapsam:
- Fonksiyon tanımı
- Limit davranışı
- Süreklilik ve türev
- Backprop’ta gradyan akışı
- ReLU / LeakyReLU ile matematiksel kıyas
- Pratik çıkarımlar

> Not: Bu defter “saf matematik” değil; **deep learning açısından gerekli matematik**.

## 1) Tanım

SiLU (Swish) fonksiyonu:

\[
\mathrm{SiLU}(x) = x \cdot \sigma(x)
\]

Burada sigmoid:

\[
\sigma(x) = \frac{1}{1 + e^{-x}}
\]

SiLU iki parçalı değildir; **her yerde tek bir ifadeyle** tanımlanır. Bu yüzden:
- süreklilik, türev, gradyan akışı konuları daha “temiz” çıkar.

## 2) Temel limit davranışı (sezgi)

Sigmoid’in limitleri:

\[
\lim_{x \to +\infty} \sigma(x) = 1,
\quad
\lim_{x \to -\infty} \sigma(x) = 0
\]

Bunları SiLU’ya uygularsak:

- \(x \to +\infty\) iken:
\[
\mathrm{SiLU}(x) = x \sigma(x) \approx x \cdot 1 = x
\]
→ Pozitif büyük değerlerde **lineer** davranır (ReLU benzeri).

- \(x \to -\infty\) iken:
\[
\mathrm{SiLU}(x) = x \sigma(x) \approx x \cdot 0 = 0
\]
→ Negatif büyük değerlerde **0’a yaklaşır** (ama ReLU gibi keskin değil).

## 3) Süreklilik ve türevlenebilirlik

- \(\sigma(x)\) her yerde süreklidir ve türevlenebilir.
- \(x\) her yerde süreklidir ve türevlenebilir.
- Çarpımları olan \(\mathrm{SiLU}(x) = x\sigma(x)\) da **her yerde süreklidir ve türevlenebilir**.

Bu, ReLU’ya göre büyük farktır:
- ReLU’da \(x=0\) noktasında “köşe” vardır (klasik türev yok).
- SiLU’da böyle bir köşe yoktur.

## 4) Türev: adım adım

\[
\mathrm{SiLU}(x) = x \sigma(x)
\]

Çarpım kuralı:

\[
\frac{d}{dx}\mathrm{SiLU}(x)
=
\frac{d}{dx}\big(x\sigma(x)\big)
=
\sigma(x) + x \sigma'(x)
\]

Sigmoid’in türevi:

\[
\sigma'(x) = \sigma(x)\big(1 - \sigma(x)\big)
\]

Yerine koyarsak:

\[
\frac{d}{dx}\mathrm{SiLU}(x)
=
\sigma(x) + x\sigma(x)\big(1-\sigma(x)\big)
\]

Bu en yaygın yazılıştır.

## 5) Türevin işareti ve “neden bazen negatif bölgede de öğrenir?”

Türevi:

\[
\mathrm{SiLU}'(x) = \sigma(x) + x\sigma(x)(1-\sigma(x))
\]

Buradan şu çıkar:
- \(\sigma(x) > 0\) her yerde
- İkinci terim \(x\)’e bağlı olduğu için negatif bölgede türev:
  - **küçük ama sıfır olmayan** değerler alır
  - hatta bazı aralıklarda türev negatif bile olabilir (Swish'in “non-monotonic” özelliği)

Pratik yorum:
- ReLU negatifte **tam 0 türev** ile gradyanı kesebilir.
- SiLU negatifte gradyanı “yumuşak” şekilde azaltır, genelde tamamen öldürmez.

## 6) Backprop: “Buradan dolayı bu oldu”

Bir katmanda:

\[
z = Wx + b
\quad,\quad
a = \mathrm{SiLU}(z)
\quad,\quad
\mathcal{L} = \mathcal{L}(a)
\]

Zincir kuralı:

\[
\frac{\partial \mathcal{L}}{\partial z}
=
\frac{\partial \mathcal{L}}{\partial a}
\cdot
\frac{da}{dz}
=
\delta \cdot \mathrm{SiLU}'(z)
\]

Burada:

\[
\delta = \frac{\partial \mathcal{L}}{\partial a}
\]

### Sonuç
- Gradyanın büyüklüğü doğrudan \(\mathrm{SiLU}'(z)\) ile ölçeklenir.
- \(\mathrm{SiLU}'(z)\) süreklidir → eğitimde “ani sıçramalar” daha az olabilir.

## 7) ReLU / LeakyReLU / SiLU (matematiksel kıyas)

### ReLU
\[
\mathrm{ReLU}(x)=\max(0,x)
\]
- Parçalı
- \(x=0\) noktasında klasik türev yok
- Negatifte türev 0 → gradyan kesilebilir

### LeakyReLU
\[
\mathrm{LReLU}(x)=\begin{cases}
x, & x\ge 0\\
\alpha x, & x<0
\end{cases}
\]
- Parçalı
- Negatifte türev \(\alpha\) → gradyan tamamen kesilmez
- \(\alpha\) sabittir

### SiLU
\[
\mathrm{SiLU}(x)=x\sigma(x)
\]
- Tek ifade
- Her yerde türevlenebilir (smooth)
- Negatifte gradyan “yumuşak” azalır
- Sabit bir eğim değil, **x’e bağlı** bir ölçekleme vardır

## 8) Pratik çıkarımlar (matematikten doğan)

Matematik şunu söylüyor:

- SiLU’nun türevi **sürekli** → optimizasyonda “kırılma” daha az
- Negatif bölgede gradyan genelde tamamen sıfırlanmaz → dead ReLU riskini azaltabilir
- Pozitif büyük değerlerde lineere yaklaşır → güçlü aktivasyon taşır
- Ancak sigmoid içerdiği için hesaplama maliyeti ReLU’dan yüksektir

Bu yüzden SiLU:
- modern CNN/detection mimarilerinde sık tercih edilir
- ama mobil/edge gibi hız kritik yerlerde ReLU/LeakyReLU seçilebilir

## 9) Mini doğrulama (türevi otomatik türev ile kontrol)

Aşağıdaki küçük demo, türev formülünün autograd ile uyumlu olduğunu gösterir.

In [1]:
import torch

def silu(x):
    return x * torch.sigmoid(x)

x = torch.tensor([-3.0, -1.0, 0.0, 1.0, 3.0], requires_grad=True)
y = silu(x).sum()
y.backward()
autograd_grad = x.grad.detach().clone()

# Analitik türev: σ(x) + xσ(x)(1-σ(x))
x2 = x.detach()
sigma = torch.sigmoid(x2)
analytic_grad = sigma + x2 * sigma * (1 - sigma)

print("x             :", x2.tolist())
print("autograd grad :", autograd_grad.tolist())
print("analytic grad :", analytic_grad.tolist())
print("max abs diff  :", (autograd_grad - analytic_grad).abs().max().item())

x             : [-3.0, -1.0, 0.0, 1.0, 3.0]
autograd grad : [-0.08810410648584366, 0.07232949137687683, 0.5, 0.9276705384254456, 1.0881041288375854]
analytic grad : [-0.08810412138700485, 0.07232949137687683, 0.5, 0.9276705384254456, 1.0881041288375854]
max abs diff  : 1.4901161193847656e-08
